In [1]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import os
import glob
import sys


# Importing callbacks and data augmentation utils
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import  Adam
import scipy.io as sio

In [6]:
from sklearn.metrics import mean_squared_error
import os
import os
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
class VolumeWiseNMSE(tf.keras.callbacks.Callback):
    def __init__(self, val_files, save_best_path):
        self.val_files = val_files
        self.best_nmse = float('inf')
        self.save_best_path = save_best_path

    def on_epoch_end(self, epoch, logs=None):
        total_nmse = 0
        num_volumes = 0

        for file_path in self.val_files:
            with h5py.File(file_path, 'r') as f:
                image_under = f['image_under'][:]  # [slices, H, W, 2]
                gt = f['image_full'][:]

            pred = []
            for i in range(image_under.shape[0]):
                input_slice = np.expand_dims(image_under[i], axis=0)
                pred_slice = self.model.predict(input_slice, verbose=0)
                pred.append(pred_slice[0])
            pred = np.stack(pred)

            nmse = np.sum((np.abs(gt - pred) ** 2)) / np.sum((np.abs(gt) ** 2))
            total_nmse += nmse
            num_volumes += 1

        avg_nmse = total_nmse / num_volumes
        print(f"\nEpoch {epoch + 1} - Avg NMSE: {avg_nmse:.6f}")

        if avg_nmse < self.best_nmse:
            print(f"New best model found! Saving model with NMSE {avg_nmse:.6f}")
            self.best_nmse = avg_nmse
            self.model.save(self.save_best_path)


In [3]:
class MRISliceGenerator(tf.keras.utils.Sequence):
    def __init__(self, file_list, batch_size=4, shuffle=True):
        self.file_list = file_list
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.slice_index_map = []  # list of (file_idx, slice_idx)
        self._build_index()

    def _build_index(self):
        for file_idx, file_path in enumerate(self.file_list):
            with h5py.File(file_path, 'r') as f:
                num_slices = f['image_under'].shape[0]
                for slice_idx in range(num_slices):
                    self.slice_index_map.append((file_idx, slice_idx))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.slice_index_map) / self.batch_size))

    def __getitem__(self, index):
        batch_map = self.slice_index_map[index * self.batch_size:(index + 1) * self.batch_size]
        batch_x, batch_y = [], []

        for file_idx, slice_idx in batch_map:
            with h5py.File(self.file_list[file_idx], 'r') as f:
                x = f['image_under'][slice_idx]  # [H, W, 2]
                y = f['image_full'][slice_idx]   # [H, W, 2]

                #y_norm = complex_zscore(y)

                batch_x.append(x)
                batch_y.append(y)

        return np.array(batch_x), np.array(batch_y)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.slice_index_map)

In [10]:
import h5py
import numpy as np
import glob
import os

val_folder = r"D:\fastmri_singlecoil_FSSCAN\val_norm"
#kspace_files_list_train = sorted(glob.glob(os.path.join(train_folder, "*.h5")))
kspace_files_list_val = sorted(glob.glob(os.path.join(val_folder, "*.h5")))
kspace_files_list_val

['D:\\fastmri_singlecoil_FSSCAN\\val_norm\\volume_000.h5',
 'D:\\fastmri_singlecoil_FSSCAN\\val_norm\\volume_001.h5',
 'D:\\fastmri_singlecoil_FSSCAN\\val_norm\\volume_002.h5',
 'D:\\fastmri_singlecoil_FSSCAN\\val_norm\\volume_003.h5',
 'D:\\fastmri_singlecoil_FSSCAN\\val_norm\\volume_004.h5',
 'D:\\fastmri_singlecoil_FSSCAN\\val_norm\\volume_005.h5',
 'D:\\fastmri_singlecoil_FSSCAN\\val_norm\\volume_006.h5',
 'D:\\fastmri_singlecoil_FSSCAN\\val_norm\\volume_007.h5',
 'D:\\fastmri_singlecoil_FSSCAN\\val_norm\\volume_008.h5',
 'D:\\fastmri_singlecoil_FSSCAN\\val_norm\\volume_009.h5',
 'D:\\fastmri_singlecoil_FSSCAN\\val_norm\\volume_010.h5',
 'D:\\fastmri_singlecoil_FSSCAN\\val_norm\\volume_011.h5',
 'D:\\fastmri_singlecoil_FSSCAN\\val_norm\\volume_012.h5',
 'D:\\fastmri_singlecoil_FSSCAN\\val_norm\\volume_013.h5',
 'D:\\fastmri_singlecoil_FSSCAN\\val_norm\\volume_014.h5',
 'D:\\fastmri_singlecoil_FSSCAN\\val_norm\\volume_015.h5',
 'D:\\fastmri_singlecoil_FSSCAN\\val_norm\\volume_016.h5

In [ ]:
# half_train = 20
# half_val = 10
#half_train = len(kspace_files_list_train) 
half_val = len(kspace_files_list_val) 
# print("half_train",half_train)
# print("half_val",half_val)
#kspace_files_list_train = kspace_files_list_train[:]
kspace_files_list_val = kspace_files_list_val[:]

# Create generators
# train_gen = MRISliceGenerator(kspace_files_list_train,batch_size=16, shuffle=True,mask=mask)
# val_gen = MRISliceGenerator(kspace_files_list_val, batch_size=4, shuffle=False,mask=mask)
#train_gen = MRISliceGenerator(kspace_files_list_train,batch_size=8, shuffle=True)
val_gen = MRISliceGenerator(kspace_files_list_val, batch_size=4, shuffle=False)

#print(len(train_gen))  
print(len(val_gen))  

NameError: name 'MRISliceGenerator' is not defined

In [4]:
%run model_unet.ipynb
H, W = 320, 320
model = unet(H=H, W=W, channels=2, kshape=(3, 3))

In [7]:
H, W = 320, 320
model = unet(H=H, W=W, channels=2, kshape=(3, 3))
opt = RMSprop(learning_rate=0.001)
def nmse(gt: np.ndarray, pred: np.ndarray) -> np.ndarray:
    """Compute Normalized Mean Squared Error (NMSE) over full volume."""
    return np.linalg.norm(gt - pred) ** 2 / np.linalg.norm(gt) ** 2

model_name = "./SavedModels_FULL_SET/best_model_nmse_full_volume.h5"
model.compile(loss='mae', optimizer=opt, metrics=[nmse])
#SOTA_paper_2_Unet/SavedModels_FULL_SET
# Load weights if available
if os.path.isfile(model_name):
    model.load_weights(model_name)
    print("Weights loaded")


Weights loaded


In [6]:
import os
import time
import glob
import psutil
import numpy as np
import h5py
from tqdm import tqdm
import tensorflow as tf
from keras_flops import get_flops

# ============================================================
# CONFIGURATION
# ============================================================
VAL_FOLDER = r"D:\fastmri_singlecoil_FSSCAN\val_norm"
WARMUP_SLICES = 10
NUM_TIMING_SLICES = 100   # fixed number of slices for benchmarking

# ============================================================
# FILE LIST
# ============================================================
file_paths = sorted(glob.glob(os.path.join(VAL_FOLDER, "*.h5")))

# ============================================================
# MODEL MUST BE LOADED
# ============================================================
# model = ...
assert model is not None, "Model is not loaded."

# ============================================================
# PARAMETER COUNT
# ============================================================
num_params = model.count_params()

# ============================================================
# FLOPs (PER SLICE, BATCH SIZE = 1)
# ============================================================
flops = get_flops(model, batch_size=1)

# ============================================================
# MEMORY HELPERS
# ============================================================
process = psutil.Process(os.getpid())

def cpu_memory_mb():
    return process.memory_info().rss / (1024 ** 2)

def gpu_memory_mb():
    info = tf.config.experimental.get_memory_info("GPU:0")
    return info["peak"] / (1024 ** 2)

# ============================================================
# LATENCY / THROUGHPUT MEASUREMENT
# ============================================================
def measure_latency(device):

    latencies = []

    with tf.device(device):

        # -----------------------------
        # WARM-UP
        # -----------------------------
        for file in file_paths[:1]:
            with h5py.File(file, "r") as f:
                image_under = f["image_under"][:]

            for s in range(min(WARMUP_SLICES, image_under.shape[0])):
                _ = model(image_under[s:s+1], training=False)

        # -----------------------------
        # TIMED INFERENCE
        # -----------------------------
        count = 0
        for file in tqdm(file_paths, desc=f"Timing on {device}", ncols=120):

            with h5py.File(file, "r") as f:
                image_under = f["image_under"][:]

            for s in range(image_under.shape[0]):

                if count >= NUM_TIMING_SLICES:
                    break

                slice_input = image_under[s:s+1]   # (1, H, W, 2)
                assert slice_input.shape[0] == 1

                start = time.perf_counter()
                _ = model(slice_input, training=False)

                # 🔑 GPU synchronization
                if "GPU" in device:
                    tf.config.experimental.get_memory_info("GPU:0")

                end = time.perf_counter()

                latencies.append(end - start)
                count += 1

            if count >= NUM_TIMING_SLICES:
                break

    latencies = np.array(latencies)

    mean_s = latencies.mean()
    median_s = np.median(latencies)

    return {
        "mean_s": mean_s,
        "median_s": median_s,
        "std_s": latencies.std(),
        "slices_per_sec": 1.0 / mean_s
    }

# ============================================================
# CPU BENCHMARK
# ============================================================
cpu_mem_before = cpu_memory_mb()
cpu_latency = measure_latency("/CPU:0")
cpu_mem_after = cpu_memory_mb()
cpu_mem_peak = cpu_mem_after - cpu_mem_before

# ============================================================
# GPU BENCHMARK (IF AVAILABLE)
# ============================================================
gpu_latency = None
gpu_mem_peak = None
gpu_name = None

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    tf.config.experimental.reset_memory_stats("GPU:0")
    gpu_name = tf.config.experimental.get_device_details(gpus[0])["device_name"]
    gpu_latency = measure_latency("/GPU:0")
    gpu_mem_peak = gpu_memory_mb()

# ============================================================
# FINAL REPORT
# ============================================================
print("\n" + "=" * 70)
print("MODEL EFFICIENCY REPORT — UNet (BATCH SIZE = 1)")
print("=" * 70)

print(f"Parameters: {num_params / 1e6:.2f} M")
print(f"FLOPs:      {flops / 1e9:.2f} GFLOPs (per slice)")

print("\n--- CPU Inference ---")
print(f"Latency:    {cpu_latency['mean_s']:.2f} s / slice")
print(f"Throughput:{cpu_latency['slices_per_sec']:.3f} slices/sec")
print(f"Memory:     {cpu_mem_peak:.2f} MB")

if gpu_latency:
    print("\n--- GPU Inference ---")
    print(f"GPU:        {gpu_name}")
    print(f"Latency:    {gpu_latency['mean_s']:.2f} s / slice")
    print(f"Throughput:{gpu_latency['slices_per_sec']:.3f} slices/sec")
    print(f"Peak VRAM:  {gpu_mem_peak:.2f} MB")
else:
    print("\nGPU not available.")

print("=" * 70)


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Timing on /GPU:0:   1%|▋                                                                | 2/199 [00:05<08:43,  2.66s/it]


MODEL EFFICIENCY REPORT — UNet (BATCH SIZE = 1)
Parameters: 8.63 M
FLOPs:      39.80 GFLOPs (per slice)

--- CPU Inference ---
Latency:    0.33 s / slice
Throughput:3.033 slices/sec
Memory:     2.91 MB

--- GPU Inference ---
GPU:        NVIDIA RTX A5000
Latency:    0.05 s / slice
Throughput:18.995 slices/sec
Peak VRAM:  195.44 MB


In [3]:

import numpy as np

def count_parameters_millions(model):
    trainable = np.sum([np.prod(v.shape) for v in model.trainable_variables])
    non_trainable = np.sum([np.prod(v.shape) for v in model.non_trainable_variables])
    total = trainable + non_trainable
    return (
        total / 1e6,
        trainable / 1e6,
        non_trainable / 1e6
    )

total_M, trainable_M, non_trainable_M = count_parameters_millions(model)

print("\n" + "=" * 40)
print(f"Total parameters:       {total_M:.3f} M")
print(f"Trainable parameters:   {trainable_M:.3f} M")
print(f"Non-trainable params:   {non_trainable_M:.3f} M")
print("=" * 40)
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

def compute_flops(model, input_shape):
    """
    input_shape: tuple, e.g. (1, H, W, 2)
    returns FLOPs (float operations) for one forward pass
    """

    @tf.function
    def forward(x):
        return model(x)

    concrete_func = forward.get_concrete_function(
        tf.TensorSpec(input_shape, tf.float32)
    )

    frozen_func = convert_variables_to_constants_v2(concrete_func)
    graph_def = frozen_func.graph.as_graph_def()

    with tf.Graph().as_default() as graph:
        tf.graph_util.import_graph_def(graph_def, name="")

        run_meta = tf.compat.v1.RunMetadata()
        opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

        flops = tf.compat.v1.profiler.profile(
            graph=graph,
            run_meta=run_meta,
            cmd="op",
            options=opts
        )

    return flops.total_float_ops
# Example: infer H, W from your data or define explicitly
H, W = 320,320

flops = compute_flops(model, input_shape=(1, H, W, 2))

print(f"FLOPs (single forward pass): {flops / 1e9:.2f} GFLOPs")



Total parameters:       8.630 M
Trainable parameters:   8.630 M
Non-trainable params:   0.000 M
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
FLOPs (single forward pass): 39.80 GFLOPs


In [16]:
import tensorflow.keras.backend as K
from tqdm import tqdm
def nmse(y_true, y_pred):
    return K.mean(K.square(y_true - y_pred)) / K.mean(K.square(y_true))

In [17]:
H, W = 320, 320
model = unet(H=H, W=W, channels=2, kshape=(3, 3))
opt = RMSprop(learning_rate=0.001)

model_name = "./SavedModels_FULL_SET/best_model_nmse_full_volume.h5"
model.compile(loss='mae', optimizer=opt, metrics=[nmse])
#SOTA_paper_2_Unet/SavedModels_FULL_SET
# Load weights if available
if os.path.isfile(model_name):
    model.load_weights(model_name)
    print("Weights loaded")


Weights loaded


In [11]:
pd_files = []
pdfs_files = []

for f in kspace_files_list_val:
    if "PDFS" in f:
        pdfs_files.append(f)
    else:
        pd_files.append(f)

print(f"PD volumes: {len(pd_files)}")
print(f"PDFS volumes: {len(pdfs_files)}")


PD volumes: 100
PDFS volumes: 99


In [14]:
import numpy as np
import h5py
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import os
import pandas as pd

import math
 # <-- Required for plotting
from typing import Optional
from skimage.metrics import peak_signal_noise_ratio

from skimage.metrics import structural_similarity as compare_ssim
from tqdm import tqdm

def psnr(
    gt: np.ndarray, pred: np.ndarray, maxval: Optional[float] = None
) -> np.ndarray:
    """Compute Peak Signal to Noise Ratio metric (PSNR)"""
    if maxval is None:
        maxval = gt.max()
    return peak_signal_noise_ratio(gt, pred, data_range=maxval)

def nmse(gt: np.ndarray, pred: np.ndarray) -> np.ndarray:
    """Compute Normalized Mean Squared Error (NMSE) over full volume."""
    return np.linalg.norm(gt - pred) ** 2 / np.linalg.norm(gt) ** 2

def visualize_and_save_all_slices(model, val_files, excel_path="validation_metrics_UNet.xlsx"):

    all_volume_psnrs = []
    nmse_all_volumes = []
    ssim_all_volumes = []

    volume_metrics = []   # Sheet 1
    slice_metrics  = []   # Sheet 2

    for file_path in tqdm(pdfs_files, desc="Processing volumes", ncols=120):

        volume_name = os.path.splitext(os.path.basename(file_path))[0]

        with h5py.File(file_path, 'r') as f:
            image_under = f['image_under'][:]   # [S, H, W, 2]
            gt          = f['image_full'][:]    # [S, H, W, 2]
            max_val     = float(f["max_val_full_image"][0])

        # ----------------------
        # Inference (slice-wise)
        # ----------------------
        pred = []
        for i in range(image_under.shape[0]):
            input_slice = np.expand_dims(image_under[i], axis=0)
            pred_slice = model.predict(input_slice, verbose=0)
            pred.append(pred_slice[0])
        pred = np.stack(pred)  # [S, H, W, 2]

        # ----------------------
        # Volume-wise PSNR
        pred=pred*max_val
        gt=gt*max_val
        # ----------------------
        volume_psnr = psnr(gt, pred, maxval=max_val)
        all_volume_psnrs.append(volume_psnr)

        # ----------------------
        # Magnitude images
        # ----------------------
        gt_complex   = gt[..., 0] + 1j * gt[..., 1]
        pred_complex = pred[..., 0] + 1j * pred[..., 1]

        mag_gt   = np.abs(gt_complex)
        mag_pred = np.abs(pred_complex)

        # ----------------------
        # Volume-wise NMSE
        # ----------------------
        nmse_volume = nmse(mag_gt.flatten(), mag_pred.flatten())
        nmse_all_volumes.append(nmse_volume)

        # ----------------------
        # Slice-wise SSIM
        # ----------------------
        ssim_slices = []
        for i in range(mag_gt.shape[0]):
            ssim_slice = compare_ssim(
                mag_gt[i],
                mag_pred[i],
                data_range=max_val,
                win_size=7,
                gaussian_weights=False,
                use_sample_covariance=False,
                K1=0.01,
                K2=0.03,
                full=False
            )

            ssim_slices.append(ssim_slice)

            # Save slice-wise
            slice_metrics.append({
                "Volume": volume_name,
                "Slice_ID": i,
                "SSIM": ssim_slice
            })

        ssim_volume = np.mean(ssim_slices)
        ssim_all_volumes.append(ssim_volume)

        # Save volume-wise
        volume_metrics.append({
            "Volume": volume_name,
            "PSNR (dB)": volume_psnr,
            "NMSE": nmse_volume,
            "SSIM (mean over slices)": ssim_volume
        })

    # ----------------------
    # Print summary
    # ----------------------
    print("\n" + "=" * 40)
    print(f"Overall PSNR: {np.mean(all_volume_psnrs):.6f} ± {np.std(all_volume_psnrs):.6f} dB")
    print(f"Overall NMSE: {np.mean(nmse_all_volumes):.6f} ± {np.std(nmse_all_volumes):.6f}")
    print(f"Overall SSIM: {np.mean(ssim_all_volumes):.6f} ± {np.std(ssim_all_volumes):.6f}")
    print("=" * 40)

    # ----------------------
    # Save to Excel
    # ----------------------
    df_volume = pd.DataFrame(volume_metrics)
    df_slice  = pd.DataFrame(slice_metrics)

    # with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
    #     df_volume.to_excel(writer, sheet_name="Volume_PSNR_NMSE_SSIM", index=False)
    #     df_slice.to_excel(writer, sheet_name="Slice_SSIM", index=False)

    print(f"\nMetrics saved to: {excel_path}")


In [15]:

output_dir = "./validation_results"
visualize_and_save_all_slices(model, kspace_files_list_val)

Processing volumes: 100%|███████████████████████████████████████████████████████████████| 99/99 [07:00<00:00,  4.25s/it]


Overall PSNR: 30.634503 ± 3.100962 dB
Overall NMSE: 0.028227 ± 0.012800
Overall SSIM: 0.767543 ± 0.079443

Metrics saved to: validation_metrics_UNet.xlsx


In [11]:
import numpy as np
import h5py
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import os

import math
 # <-- Required for plotting
from typing import Optional
from skimage.metrics import peak_signal_noise_ratio

from skimage.metrics import structural_similarity as compare_ssim
from tqdm import tqdm

def psnr(
    gt: np.ndarray, pred: np.ndarray, maxval: Optional[float] = None
) -> np.ndarray:
    """Compute Peak Signal to Noise Ratio metric (PSNR)"""
    if maxval is None:
        maxval = gt.max()
    return peak_signal_noise_ratio(gt, pred, data_range=maxval)

def nmse(gt: np.ndarray, pred: np.ndarray) -> np.ndarray:
    """Compute Normalized Mean Squared Error (NMSE) over full volume."""
    return np.linalg.norm(gt - pred) ** 2 / np.linalg.norm(gt) ** 2

#
def visualize_and_save_all_slices(model, val_files):
    #os.makedirs(output_dir, exist_ok=True)

    all_volume_psnrs = []
    nmse_all_volumes = []
    ssim_all_volumes=[]

    for file_path in val_files:
        volume_name = os.path.splitext(os.path.basename(file_path))[0]
        #pdf_path = os.path.join(output_dir, f"{volume_name}.pdf")

        with h5py.File(file_path, 'r') as f:
            image_under = f['image_under'][:]  # [slices, H, W, 2]
            gt = f['image_full'][:]            # [slices, H, W, 2]
            max_val  = float(f["max_val_full_image"][0])

        pred = []
        for i in range(image_under.shape[0]):
            input_slice = np.expand_dims(image_under[i], axis=0)  # [1, H, W, 2]
            pred_slice = model.predict(input_slice, verbose=0)
            pred.append(pred_slice[0])
        pred = np.stack(pred)  # [slices, H, W, 2]

        # Volume-wise PSNR using all channels
        #pred=pred*max_val
        #gt=gt*max_val
        volume_psnr = psnr(gt, pred,maxval=max_val)
        all_volume_psnrs.append(volume_psnr)

        # Convert complex for visualization (abs)
        gt_complex = gt[..., 0] + 1j * gt[..., 1]
        pred_complex = pred[..., 0] + 1j * pred[..., 1]
        under_complex = image_under[..., 0] + 1j * image_under[..., 1]

        mag_gt = np.abs(gt_complex)
        mag_pred = np.abs(pred_complex)
        mag_under = np.abs(under_complex)
                # Compute NMSE over full volume (flattened)
        nmse_volume = nmse(mag_gt.flatten(), mag_pred.flatten())
    
        nmse_all_volumes.append(nmse_volume)

        
        #max_val=np.max(mag_gt)
    
        ssim_slices = []
        for i in range(mag_gt.shape[0]):
            gt_mag = mag_gt[i]
            pred_mag = mag_pred[i]
    
            ssim_slice= compare_ssim(
                gt_mag, pred_mag,
                data_range=max_val,
                win_size=7,
                gaussian_weights=False,
                use_sample_covariance=False,      # <-- now correct per paper
                K1=0.01,
                K2=0.03,
                full=False
            )
            ssim_slices.append(ssim_slice)
    
        ssim_volume = np.mean(ssim_slices)
        ssim_all_volumes.append(ssim_volume)
    


        
    overall_avg_psnr = np.mean(all_volume_psnrs)
    overall_std_psnr = np.std(all_volume_psnrs)
    print("\n" + "="*40)
    print(f"Overall Average PSNR: {overall_avg_psnr:.6f} ± {overall_std_psnr:.6f} dB")
    print("="*40)

    
    overall_avg_nmse = np.mean(nmse_all_volumes)
    overall_std_nmse = np.std(nmse_all_volumes)
    print("\n" + "="*40)
    print(f"Overall Average NMSE: {overall_avg_nmse:.6f} ± {overall_std_nmse:.6f} dB")
    print("="*40)

    overall_avg_ssim = np.mean(ssim_all_volumes)
    overall_std_ssim = np.std(ssim_all_volumes)
    print("\n" + "="*40)
    print(f"Overall Average SSIM: {overall_avg_ssim:.6f} ± {overall_std_ssim:.6f} dB")
    print("="*40)


NameError: name 'tqdm' is not defined

In [10]:
import numpy as np
import h5py
import tensorflow as tf
import os
from typing import Optional
from tqdm import tqdm

from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity as compare_ssim


# ============================================================
# Metrics
# ============================================================

def psnr(
    gt: np.ndarray, pred: np.ndarray, maxval: Optional[float] = None
) -> np.ndarray:
    """Compute Peak Signal to Noise Ratio metric (PSNR)"""
    if maxval is None:
        maxval = gt.max()
    return peak_signal_noise_ratio(gt, pred, data_range=maxval)


def nmse(gt: np.ndarray, pred: np.ndarray) -> np.ndarray:
    """Compute Normalized Mean Squared Error (NMSE) over full volume."""
    return np.linalg.norm(gt - pred) ** 2 / np.linalg.norm(gt) ** 2


# ============================================================
# UNet inference (RUN ONCE + SAVE IN H5)
# ============================================================

def get_or_create_unet_prediction(model, h5_path):
    """
    If UNet prediction exists → load it
    Else → run inference, save it, return it
    """
    with h5py.File(h5_path, "a") as f:

        # If prediction already exists
        if "image_unet_pred" in f:
            print(f"✔ Loading saved UNet prediction: {os.path.basename(h5_path)}")
            return f["image_unet_pred"][:]

        # Else run inference
        print(f"⏳ Running UNet inference: {os.path.basename(h5_path)}")
        image_under = f["image_under"][:]   # [S, H, W, 2]

        preds = []
        for i in range(image_under.shape[0]):
            inp = np.expand_dims(image_under[i], axis=0)  # [1, H, W, 2]
            out = model.predict(inp, verbose=0)
            preds.append(out[0])

        pred = np.stack(preds).astype(np.float32)

        # Save prediction
        f.create_dataset(
            "image_unet_pred",
            data=pred,
            dtype="float32",
            compression="gzip"
        )

        # Optional metadata
        f.attrs["prediction_model"] = "UNet"

        print(f"💾 Saved UNet prediction in H5")

        return pred


# ============================================================
# MAIN EVALUATION FUNCTION
# ============================================================

def visualize_and_save_all_slices(model, val_files):

    all_volume_psnrs = []
    nmse_all_volumes = []
    ssim_all_volumes = []

    for file_path in tqdm(val_files, desc="Evaluating volumes"):

        with h5py.File(file_path, 'r') as f:
            image_under = f['image_under'][:]  # [S, H, W, 2]
            gt = f['image_full'][:]            # [S, H, W, 2]

        # ----------------------------------------------------
        # Load or compute UNet prediction
        # ----------------------------------------------------
        pred = get_or_create_unet_prediction(model, file_path)

        # ----------------------------------------------------
        # PSNR (volume-wise, both channels)
        # ----------------------------------------------------
        volume_psnr = psnr(gt, pred)
        all_volume_psnrs.append(volume_psnr)

        # ----------------------------------------------------
        # Convert to complex magnitude
        # ----------------------------------------------------
        gt_complex = gt[..., 0] + 1j * gt[..., 1]
        pred_complex = pred[..., 0] + 1j * pred[..., 1]

        mag_gt = np.abs(gt_complex)
        mag_pred = np.abs(pred_complex)

        # ----------------------------------------------------
        # NMSE (full volume)
        # ----------------------------------------------------
        nmse_volume = nmse(mag_gt.flatten(), mag_pred.flatten())
        nmse_all_volumes.append(nmse_volume)

        # ----------------------------------------------------
        # SSIM (slice-wise → volume average)
        # ----------------------------------------------------
        max_val = np.max(mag_gt)
        ssim_slices = []

        for i in range(mag_gt.shape[0]):
            ssim_slice = compare_ssim(
                mag_gt[i],
                mag_pred[i],
                data_range=max_val,
                win_size=7,
                gaussian_weights=False,
                use_sample_covariance=False,
                K1=0.01,
                K2=0.03
            )
            ssim_slices.append(ssim_slice)

        ssim_volume = np.mean(ssim_slices)
        ssim_all_volumes.append(ssim_volume)

    # ========================================================
    # FINAL RESULTS
    # ========================================================

    print("\n" + "=" * 50)
    print(
        f"Overall Average PSNR : {np.mean(all_volume_psnrs):.6f} ± "
        f"{np.std(all_volume_psnrs):.6f} dB"
    )

    print(
        f"Overall Average NMSE : {np.mean(nmse_all_volumes):.6f} ± "
        f"{np.std(nmse_all_volumes):.6f}"
    )

    print(
        f"Overall Average SSIM : {np.mean(ssim_all_volumes):.6f} ± "
        f"{np.std(ssim_all_volumes):.6f}"
    )
    print("=" * 50)


In [11]:

output_dir = "./validation_results"
visualize_and_save_all_slices(model, kspace_files_list_val)

Evaluating volumes:   0%|                                                                      | 0/972 [00:00<?, ?it/s]

⏳ Running UNet inference: volume_000.h5
💾 Saved UNet prediction in H5


Evaluating volumes:   0%|                                                            | 1/972 [00:08<2:14:57,  8.34s/it]

⏳ Running UNet inference: volume_001.h5
💾 Saved UNet prediction in H5


Evaluating volumes:   0%|                                                            | 2/972 [00:12<1:30:51,  5.62s/it]

...

⏳ Running UNet inference: volume_PDFS_487.h5
💾 Saved UNet prediction in H5


Evaluating volumes: 100%|█████████████████████████████████████████████████████████▉| 971/972 [1:49:36<00:10, 10.92s/it]

⏳ Running UNet inference: volume_PDFS_488.h5
💾 Saved UNet prediction in H5


Evaluating volumes: 100%|██████████████████████████████████████████████████████████| 972/972 [1:49:49<00:00,  6.78s/it]


Overall Average PSNR : 32.710713 ± 3.742883 dB
Overall Average NMSE : 0.018311 ± 0.012692
Overall Average SSIM : 0.803880 ± 0.072204
